<a href="https://www.kaggle.com/code/yaaangzhou/best-ps4-games-eda-based-on-plotly-express?scriptVersionId=133891357" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/world-populationyearly-changedensity-by-year/world_pop.csv
/kaggle/input/list-of-best-selling-ps4-games/List of best-selling PlayStation 4 video games.csv


# Imports and prepare data

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('/kaggle/input/list-of-best-selling-ps4-games/List of best-selling PlayStation 4 video games.csv')
df['Copies sold'] = df['Copies sold'].str.replace(' million', '').astype(float) / 1e6
df['Release date[a]'] = df['Release date[a]'].astype('datetime64')
df['Year'] = df['Release date[a]'].dt.year
df['Month'] = df['Release date[a]'].dt.month

pop = pd.read_csv('/kaggle/input/world-populationyearly-changedensity-by-year/world_pop.csv')
df = pd.merge(df,pop[['Year','World Population']],on='Year')


In [4]:
df.head()

,Game,Copies sold,Release date[a],Genre(s),Developer(s),Publisher(s),Year,Month,World Population
0,Marvel's Spider-Man,0.000020,2018-09-07,Action-adventure,Insomniac Games,Sony Interactive Entertainment,2018,9,"7,631,091,040"
1,God of War,0.000019,2018-04-20,Action-adventure hack and slash,Santa Monica Studio,Sony Interactive Entertainment,2018,4,"7,631,091,040"
2,Detroit: Become Human,0.000005,2018-05-25,Adventure,Quantic Dream,Sony Interactive Entertainment,2018,5,"7,631,091,040"
3,Monster Hunter: World,0.000005,2018-01-26,Action role-playing,Capcom,Capcom,2018,1,"7,631,091,040"
4,Uncharted 4: A Thief's End,0.000016,2016-05-10,Action-adventure,Naughty Dog,Sony Interactive Entertainment,2016,5,"7,464,022,049"


# Most Selling Games

In [5]:
fig = px.bar(df.sort_values('Copies sold'),
             x = 'Game', 
             y = 'Copies sold',
             color = 'Developer(s)',
             title = 'Most Selling Games',height=700,
             color_discrete_sequence=px.colors.qualitative.Pastel,
             hover_data='Copies sold')

fig.update_layout(template='plotly_white',xaxis_tickangle=30)

fig.update_xaxes(categoryorder='total descending')

fig.show()

# Pie plot

In [6]:
fig = px.pie(df,values = 'Copies sold',
             names = 'Genre(s)',
             title = 'Genre Distribution',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             )
fig.update_traces(textposition='inside', textinfo='label+percent')
fig.update_layout(template='plotly_white')
fig.show()

# Treemap: Developer's Games

In [7]:
df_tree = df[['Developer(s)','Genre(s)']].value_counts().reset_index(name='count')
df_tree.head()

,Developer(s),Genre(s),count
0,Naughty Dog,Action-adventure survival horror,2
1,CD Projekt Red,Action role-playing,2
2,Team Ninja,Action role-playing,2
3,Square Enix,Role-playing,2
4,Square Enix,Action role-playing,2


In [8]:
fig = px.treemap(df_tree,
                 path = ['Developer(s)','Genre(s)'],
                 values = 'count',               
                color_discrete_sequence=px.colors.qualitative.Pastel, hover_data=['count'],title = "Developer's Games")

fig.show()

# Treemap: Publisher's Games

In [9]:
df_tree = df[['Publisher(s)','Genre(s)']].value_counts().reset_index(name='count')
fig = px.treemap(df_tree,
                 path = ['Publisher(s)','Genre(s)'],
                 values = 'count',               
                color_discrete_sequence=px.colors.qualitative.Pastel, hover_data=['count'],title = "Publisher's Games")

fig.show()

# Time Series: Publish Games Count

In [10]:
df_count = df.groupby(['Year','Month']).agg({'Game':'count'}).reset_index()
df_count ['Date'] = pd.to_datetime(df_count[['Year', 'Month']].assign(day=1))

fig = px.line(df_count , x='Date', y='Game',title ='Time Series Publish Games Count',width = 600,height=400,text = 'Game',)
fig.show()